# MS Calculations

Notebook to crunch numbers for the MS.

by Cascade Tuholske 2020.02.23

In [1]:
#### Depdencies 
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

In [5]:
#### Regressions, no intercept addition is needed because we're using SK LEARN HERE 

def lm_func(df, col, scale):
    
    "simple linear model of a time series data, returns coef"
    
    # Get Data
    X_year = np.array(df.groupby('year')['ID_HDC_G0'].mean().index).reshape((-1, 1))
    Y_stats = np.array(df.groupby('year')[col].sum()).reshape((-1, 1))

    # Add Intercept
    X_year_2 = sm.add_constant(X_year)

    # Regress
    model = sm.OLS(Y_stats, X_year_2).fit() 
        
    coef = int(model.params[1]*scale)
    #coef = int(coef)
            
    # R2 and P
    r2 = model.rsquared_adj
    p = model.pvalues[0]
    
    return coef, round(r2, 2), round(p, 3)

In [2]:
#### Load Data

# file paths
DATA_IN = "/home/cascade/projects/UrbanHeat/data/"  # Note: Need ?dl=1 to make sure this file gets read correctly
FIG_OUT = "/home/cascade/projects/UrbanHeat/figures/"
FN_IN = 'processed/All_data_HI406_figdata.csv'
#FN_IN = '/home/cascade/projects/UrbanHeat/data/processed/oldworkflow/All_data_Raw406_es_final_pdayadd.csv'

HI_STATS = pd.read_csv(DATA_IN+FN_IN)

In [3]:
# Check it
HI_STATS.head()

,Unnamed: 0,ID_HDC_G0,year,total_days,P,P1983,P2016,people_days,people_days_heat,people_days_pop
0,0,22,1983,2,52064.452435,52064.452435,73006.671133,0.000104,0.000104,0.0
1,1,26,1983,1,194088.886834,194088.886834,268055.635628,0.000194,0.000194,0.0
2,2,27,1983,1,80540.779940,80540.779940,93335.494324,0.000081,0.000081,0.0
3,3,28,1983,9,59320.971209,59320.971209,91449.606255,0.000534,0.000534,0.0
4,4,29,1983,8,336518.836621,336518.836621,533318.453653,0.002692,0.002692,0.0


In [4]:
#### Drop 40 cities where P1983 == 40 -- see comments in Fig 2 & 3 code CPT 2020.03.03
print(len(HI_STATS))
HI_STATS = HI_STATS[HI_STATS['P1983'] > 0]
print(len(HI_STATS))

#### Drop cities with only one Tmax Day in 1983 and none else because you cannot regress them
drop_list = [2543, 2560, 3667, 3669, 6122, 6156] # city IDS
HI_STATS= HI_STATS[~HI_STATS['ID_HDC_G0'].isin(drop_list)]
print(len(HI_STATS))

392972
391612
391408


In [20]:
data

year
1983    129.172185
1984    118.641838
1985    121.763473
1986    122.285152
1987    148.688010
1988    150.189327
1989    132.949977
1990    159.222580
1991    148.600409
1992    144.284759
1993    156.908293
1994    167.530805
1995    179.716878
1996    175.381440
1997    180.892595
1998    221.487122
1999    174.817034
2000    188.635963
2001    201.161597
2002    208.165107
2003    215.373058
2004    214.547240
2005    226.441442
2006    234.600218
2007    233.438331
2008    229.223204
2009    250.671001
2010    274.827940
2011    247.504078
2012    254.894533
2013    261.693024
2014    268.369678
2015    288.384824
2016    301.672123
Name: people_days, dtype: float64

In [24]:
#### Pct Change in Poeple Days 1983 - 2016

(data.iloc[len(data) -1] - data.iloc[0]) / data.iloc[0] * 100

133.5426331807929

In [26]:
#### Pct Pday Annual Increase from Heat
coef_pdays, r2_pdays, p_pdays = lm_func(HI_STATS, 'people_days', 10**3) # regress pdays
coef_heat, r2_heat, p_heat = lm_func(HI_STATS, 'people_days_heat', 10**3) # regreas heat

print(coef_heat/coef_pdays *100)

22.9466718567536


# FIG 1 Code

In [ ]:
#### Make Figure & Save it

fig, axs = plt.subplots(1, 3, figsize = (16, 5), sharex=True)
ws = 0.25
fig.subplots_adjust(wspace=ws)

## PLOT 1 - People Days
color = 'blue'
data = HI_STATS.groupby('year')['people_days'].sum() 
years = list(data.index)
axs[0].plot(data, color = color, linestyle = ':', alpha = 0.5)
axs[0].fill_between(years, data, color = color, alpha = 0.1, label='_nolegend_')
sns.regplot(years, data, color = color, ax = axs[0], scatter = False, truncate = True)
axs[0].set_title('Total people-days', size = 15 )
axs[0].set_ylabel('People-days [billions]', fontsize = 13)
axs[0].set_xlabel('Year', fontsize = 13)

# Add text
coef, r2, p = lm_func(HI_STATS, 'people_days', 10**3)
text = str(round(coef/1000, 1))+' B increase in people-\ndays per year'# ($R^2$=' + str(r2) + ')*'
axs[0].text(0.07, .85, text, transform=axs[0].transAxes, size=12)
axs[0].text(0.00, 1.1, '(a)', transform=axs[0].transAxes, size=15 )


#2 PLOT 2 - People Days delta pop x days
color = 'red'
data = HI_STATS.groupby('year')['people_days_pop'].sum() 
axs[1].scatter(years, data, color = color, linestyle = ':', alpha = 0.3)
axs[1].plot(data, color = color, linestyle = ':', alpha = 0.3)
sns.regplot(years, data, color = color, ax = axs[1], scatter = False, truncate = True)
axs[1].set_title('Trend due to population growth', size = 15)
axs[1].set_ylabel('People-days [billions]', fontsize = 13)
axs[1].set_xlabel('Year', fontsize = 13)

# Add text
coef, r2, p = lm_func(HI_STATS, 'people_days_pop', 10**3)
text = str(round(coef/1000, 1))+' B increase in people-\ndays per year'# ($R^2$=' + str(r2) + ')*'
axs[1].text(1.25+.08, .85, text, transform=axs[0].transAxes, size=12)
axs[1].text(1.25, 1.1, '(b)', transform=axs[0].transAxes, size=15 )

## PLOT 3 - People Days due to heat
color = 'green'
data = HI_STATS.groupby('year')['people_days_heat'].sum()
pdays83 = data.get(data.index[0])
data = data - pdays83
years = list(data.index)
axs[2].scatter(years, data, color = color, linestyle = ':', alpha = 0.5)
axs[2].plot(data, color = color, linestyle = ':', alpha = 0.5)
sns.regplot(years, data, color = color, ax = axs[2], scatter = False, truncate = True)
axs[2].set_title('Trend due to urban warming', size = 15)
axs[2].set_xlabel('Year', fontsize = 13)
axs[2].set_ylabel('People-days [billions]', fontsize = 13)

# Add text
coef, r2, p = lm_func(HI_STATS, 'people_days_heat', 10**3)
text = str(round(coef/1000, 1))+' B increase in people-\ndays per year' # ($R^2$=' + str(r2) + ')*'
axs[2].text(1.25*2-0.04+.1, .85, text, transform=axs[0].transAxes, size=12,);
axs[1].text(1.25*2, 1.1, '(c)', transform=axs[0].transAxes, size=15 )

# Set y-axis
# for ax in fig.get_axes():
#     ax.label_outer()

plt.savefig(FIG_OUT+'FIG1.png', dpi = 300, bbox_inches='tight')

